In [4]:
import os 
from dotenv import load_dotenv
load_dotenv()
from agents import Agent, Runner,function_tool,trace, ItemHelpers, MessageOutputItem
import nest_asyncio
nest_asyncio.apply()
# setting up api key
os.environ["OPENAI_API_KEY"] = os.environ['OPENAI_API_KEY_MALI']
import wikipedia
import openmeteo_requests
import requests_cache
# from retry_requests import retry
import requests
from agents.extensions.visualization import draw_graph

import json 
import requests
from serpapi import GoogleSearch
api_key = os.getenv("SERPAPI_API_KEY")


from openai.types.responses import (
    ResponseFunctionCallArgumentsDeltaEvent,  # tool call streaming
    ResponseCreatedEvent,  # start of new event like tool call or final answer
)

from tools import flight_search, yelp_search, hotels_search, wikipedia_search, get_weather
from pydantic import BaseModel
from models import first_agent_output
from agents.model_settings import ModelSettings
from data_models.flight_data_models import FlightSearchResults

END OF THE SCRIPT


In [2]:
## agents

flight_agent = Agent(
    name = "Google flight information",
    instructions= """
Returns google flight information as a json. Always answer in Turkish language. Return all the given flight information

departure_id : Parameter defines the departure airport code or location kgmid. An airport code is an uppercase 3-letter code. For example, CDG is Paris Charles de Gaulle Airport and AUS is Austin-Bergstrom International Airport.

arrival_id : Parameter defines the arrival airport code or location kgmid. An airport code is an uppercase 3-letter code. You can search for it on Google Flights or IATA. For example, CDG is Paris Charles de Gaulle Airport and AUS is Austin-Bergstrom International Airport.

outbound_date : Parameter defines the outbound date. The format is YYYY-MM-DD. e.g. 2025-04-09

return_date : Parameter defines the return date. The format is YYYY-MM-DD. e.g. 2025-04-15
""",
    model = 'gpt-4o-mini',
    tools=[flight_search],
    # model_settings={"temperature": 0.0}
    # output_type = first_agent_output
    model_settings=ModelSettings(temperature= 0.0,
                                 max_tokens = 4096*2)
)

yelp_agent = Agent(
    name = "YELP SEARCH",
    instructions= """Returns YELP search term results.     
    search_term : search_term
    location : location""",
    model = 'gpt-4o-mini',
    tools=[yelp_search],
    model_settings=ModelSettings(temperature= 0.0,
                                 max_tokens = 4096*2),
)

hotel_agent = Agent(
    name = "Google hotels information",
    instructions= """Returns google hotels information as a json.     
    q : Search Query
    gl : Country""",
    model = 'gpt-4o-mini',
    tools=[hotels_search],
    model_settings=ModelSettings(temperature= 0.0,
                                 max_tokens = 4096*2),
)

wikipedia_agent = Agent(
    name = "Wikipedia search for given destination",
    instructions= """Return the wikipedia page content for the given location.""",
    model = 'gpt-4o-mini',
    tools=[wikipedia_search],
    model_settings=ModelSettings(temperature= 0.0,
                                 max_tokens = 4096*2),
)

weather_agent = Agent(name="Weather condition",
                instructions="You are a helpful assistant on determination of the temperature of given location. Location will be given as city name, convert that city to longitude and latitude",
                model='gpt-4o-mini',
                tools=[get_weather],
                model_settings=ModelSettings(temperature= 0.0,
                                 max_tokens = 4096*2),
                                )

travel_agent = Agent(
    name = "Travel assistant", 
    instructions= """You are a talented and experienced travela assistant. Your task it to give informantion about traing ticket, hotels, 
    restaurants, basically give information about steps of the travel.""",
    model='gpt-4o-mini',
    tools = [flight_search,hotels_search,yelp_search]
)

In [3]:
# with trace('total_travel_agent'):
#     result = Runner.run_sync(flight_agent, """I want to travel from Ankara to Istanbul. The outbound date is 2025-05-05 and the return date will be 2025-05-11""".strip())
#     print(result.final_output)

In [5]:
with trace('total_travel_agent_2'):
    result = Runner.run_sync(travel_agent, """I want to travel from Ankara to Istanbul. The outbound date is 2025-05-05 and the return date will be 2025-05-11. 
                             I also need a hotel to stay. I do not like smell of cigarette. If possible, I want to see the rooms where smoking is not allowed. 
                             In addition to that, I like eating sea food. I guess istanbul is very rich city interms of sea restaurants""".strip())
    print(result.final_output)

### Travel Details from Ankara to Istanbul

#### Outbound Flight
- **Departure**: Ankara (ANR)
- **Arrival**: Istanbul (IST)
- **Outbound Date**: **2025-05-05**
- **Return Date**: **2025-05-11**

**(Details for flight bookings will need to be confirmed closer to the date)**

---

#### Hotel Options in Istanbul
Here are a few hotels that have non-smoking rooms:

1. **Vespia Hotel**
   - **Rating**: ★★★★☆ (4.8/5)
   - **Price**: Starting at $78 per night
   - **Description**: Polished quarters with 3 restaurants, plus a spa with a gym & a hammam.
   - **Check-in**: 2:00 PM | **Check-out**: 12:00 PM
   - **Website**: [Vespia Hotel](http://www.vespiahotel.com.tr/)
   - ![Vespia Hotel](https://lh3.googleusercontent.com/gps-cs-s/AB5caB-A44ZcP2NaP7vVFbCLBrN4QVRDKFEn9p6FB0uCJEBHacvuh73IAK5KoX4YO6NB_1naGXQUloKcbq746zE5GTpJhteK1bgv2XqivuRhxiUUnN65_naA4DZRclsrWiAcEDGI6hB-=s287-w287-h192-n-k-no-v1)

2. **JW Marriott Hotel Istanbul Marmara Sea**
   - **Rating**: ★★★★★ (4.7/5)
   - **Price**: Starti

In [9]:
with trace('total_travel_agent_4'):
    result = Runner.run_sync(travel_agent, """I want to travel from Ankara to Istanbul. The outbound date is 2025-05-05 and the return date will be 2025-05-11.""".strip())
    print(result.final_output)

Here's your travel itinerary for the trip from Ankara to Istanbul on the specified dates:

### Flight Information
- **Outbound Date:** May 5, 2025
- **Return Date:** May 11, 2025

Unfortunately, I couldn't retrieve specific flight details due to a format issue. You may want to check platforms like [Skyscanner](https://www.skyscanner.com/) or [Kayak](https://www.kayak.com/) for the latest flights from Ankara (ESB) to Istanbul (IST/SAB) on those dates.

### Hotel Options in Istanbul

1. **Ultra Luxury Residence**
   - **Price:** $59 per night
   - **Rating:** 4.45/5
   - **Location:** Near Istanbul Airport
   - **Check-in:** 12:00 PM | **Check-out:** 2:00 PM 
   - [More Details](https://www.booking.com/hotel/tr/ultra-luxury-residence.en.html?aid=2068758) 

2. **Vespia Hotel**
   - **Price:** $78 per night
   - **Rating:** 4.8/5
   - **Location:** 5-star hotel
   - [More Details](http://www.vespiahotel.com.tr/)

3. **New Luxurious and Cozy Flat 6**
   - **Price:** $47 per night
   - **Rat

In [10]:
with trace('total_travel_agent_4'):
    result = Runner.run_sync(travel_agent, """I want to travel from Ankara to Istanbul. The outbound date is 2025-05-05 and the return date will be 2025-05-11. arrange me flight ticket""".strip())
    print(result.final_output)

It seems there are currently no available flights from Ankara to Istanbul for the dates you're interested in (May 5, 2025, for departure and May 11, 2025, for return). 

However, you can check for flights directly on [Google Flights](https://www.google.com/travel/flights?hl=en&gl=us&curr=USD&tfs=CBwQAhoeEgoyMDI1LTA1LTA1agcIARIDQU5LcgcIARIDSVNUGh4SCjIwMjUtMDUtMTFqBwgBEgNJU1RyBwgBEgNBTktCAQFIAXABmAEB&tfu=EgIIAQ).

If you need assistance with other travel arrangements, like hotels or restaurants in Istanbul, feel free to ask!


In [11]:
with trace('total_travel_agent_4'):
    result = Runner.run_sync(travel_agent, """I want to travel from Ankara(ESB) to Istanbul(SAW). The outbound date is 2025-05-05 and the return date will be 2025-05-11. arrange me flight ticket""".strip())
    print(result.final_output)

Here are some flight options for your round trip from Ankara (ESB) to Istanbul (SAW) on the specified dates:

### Outbound Flight: May 5, 2025

1. **Pegasus Airlines (PC 2675)**
   - **Departure:** 13:20 from Ankara Esenboğa (ESB)
   - **Arrival:** 14:30 at Sabiha Gökçen (SAW)
   - **Duration:** 70 minutes
   - **Price:** $49
   - **Aircraft:** Boeing 737
   - **Legroom:** 29 inches
   - **Features:** Stream media to your device, carbon emissions estimate: 52 kg
   - ![Pegasus Airlines](https://www.gstatic.com/flights/airline_logos/70px/PC.png)

2. **AJet (VF 3001)**
   - **Departure:** 10:15 from Ankara Esenboğa (ESB)
   - **Arrival:** 11:25 at Sabiha Gökçen (SAW)
   - **Duration:** 70 minutes
   - **Price:** $59
   - **Aircraft:** Boeing 737
   - **Legroom:** 29 inches
   - **Features:** In-seat USB outlet, carbon emissions estimate: 44 kg
   - ![AJet](https://www.gstatic.com/flights/airline_logos/70px/VF.png)

3. **AJet (VF 3003)**
   - **Departure:** 12:10 from Ankara Esenboğa (ESB

In [12]:
with trace('total_travel_agent_5'):
    result = Runner.run_sync(travel_agent, """
I am male young male. I want to travel around europe. What would you recommend?
""".strip())
    print(result.final_output)

Traveling around Europe is an exciting adventure! Here are some recommendations to help you plan your trip:

### Popular Destinations
1. **Paris, France**: Visit the Eiffel Tower, explore the Louvre, and enjoy the cafes.
2. **Rome, Italy**: Discover ancient history at the Colosseum, Vatican City, and indulge in Italian cuisine.
3. **Barcelona, Spain**: Check out Gaudí's architecture, relax on the beaches, and enjoy tapas.
4. **Amsterdam, Netherlands**: Explore the canals, visit the Van Gogh Museum, and enjoy vibrant nightlife.
5. **Berlin, Germany**: Experience the historical sites, diverse culture, and thriving nightlife.

### Travel Tips
- **Transportation**: Consider getting a Eurail pass for flexible train travel across many countries.
- **Accommodation**: Look for hostels, budget hotels, or Airbnb for cost-effective stays.
- **Local Cuisine**: Try local dishes at small restaurants and food markets.
- **Cultural Sites**: Research museums and historical sites ahead of time for ticke

In [26]:
with trace('total_travel_agent_5'):
    result = Runner.run_sync(travel_agent, """
I am male young male. I want to travel around europe. I want you to recommend me a travel plan?
""".strip())
    print(result.final_output)

Here's a suggested travel plan for a young male traveler looking to explore Europe, focusing on major cities known for their culture, nightlife, and attractions. This itinerary covers 10 days and includes recommendations for transportation, accommodations, and activities.

### Travel Plan: 10-Day European Adventure

#### Day 1-3: Paris, France
- **Arrival**: Fly into Charles de Gaulle Airport.
- **Accommodation**: Stay in a budget-friendly hostel or hotel in the Latin Quarter.
- **Activities**:
  - Visit the Eiffel Tower and take a Seine River cruise.
  - Explore the Louvre Museum.
  - Enjoy the nightlife in the Montmartre district.

#### Day 4-5: Amsterdam, Netherlands
- **Transportation**: Take a train from Paris to Amsterdam (about 3.5 hours).
- **Accommodation**: Book a hostel near the city center.
- **Activities**:
  - Visit the Anne Frank House and Van Gogh Museum.
  - Explore the canals by bike.
  - Experience the nightlife in the Leidseplein area.

#### Day 6-7: Berlin, Germany

In [27]:
def assemble_conversation(result, new_input):
    if result !=None:
        new_input = result.to_input_list() + [{'content': new_input,
                                                'role': 'user'}]
    else:
        new_input = new_input
    return new_input

In [28]:
new_prompt = """According to your recommendation arrange flight tickets, hotel and restaurants. The outbound date is 2025-05-05 and the return date will be 2025-05-11. And my travel will start from Ankara"""
new_input = assemble_conversation(result, new_prompt)

In [29]:
new_input

[{'content': 'I am male young male. I want to travel around europe. I want you to recommend me a travel plan?',
  'role': 'user'},
 {'id': 'msg_67f6b088a3e081928ff6f041cd9a04e3095ed2877e79cb6d',
  'content': [{'annotations': [],
    'text': "Here's a suggested travel plan for a young male traveler looking to explore Europe, focusing on major cities known for their culture, nightlife, and attractions. This itinerary covers 10 days and includes recommendations for transportation, accommodations, and activities.\n\n### Travel Plan: 10-Day European Adventure\n\n#### Day 1-3: Paris, France\n- **Arrival**: Fly into Charles de Gaulle Airport.\n- **Accommodation**: Stay in a budget-friendly hostel or hotel in the Latin Quarter.\n- **Activities**:\n  - Visit the Eiffel Tower and take a Seine River cruise.\n  - Explore the Louvre Museum.\n  - Enjoy the nightlife in the Montmartre district.\n\n#### Day 4-5: Amsterdam, Netherlands\n- **Transportation**: Take a train from Paris to Amsterdam (about 

In [30]:
result = await Runner.run(travel_agent, new_input)
print(result.final_output)

Here's your travel plan with flight tickets, hotel options, and restaurant recommendations for your trip to Paris:

### Flight Details
- **Departure City**: Ankara (ESB)
- **Destination City**: Paris (CDG)
- **Outbound Flight**: 
  - **Date**: May 5, 2025
  - **Airline**: Turkish Airlines / AJet 
  - **Departure**: 11:00 AM from ESB
  - **Arrival**: 4:00 PM at CDG
  - **Layover**: Istanbul (IST) for 2 hours
  - **Total Duration**: Approximately 6 hours
  - **Price**: €474 
- **Return Flight**: 
  - **Date**: May 11, 2025
  - **Airline**: Turkish Airlines / AJet
  - **Departure**: 1:20 PM from CDG
  - **Arrival**: 6:10 PM at ESB
  - **Layover**: Istanbul (IST) for 1 hour 30 minutes
  - **Total Duration**: Approximately 5 hours
  - **Price**: Included in the outbound price

**[Book Flights Here](https://www.google.com/travel/flights?hl=en&gl=us&curr=USD&tfs=CBwQAhoeEgoyMDI1LTA1LTA1agcIARIDRVNCcgcIARIDQ0RHGh4SCjIwMjUtMDUtMTFqBwgBEgNDREdyBwgBEgNFU0JCAQFIAXABmAEB&tfu=EgIIAQ)**

### Hotel Op

In [31]:
new_prompt = """what about other destinations?"""
new_input = assemble_conversation(result, new_prompt)

In [33]:
result = await Runner.run(travel_agent, new_input)
print(result.final_output)

Here’s a proposed itinerary for your traveling around Europe, along with flight, hotel, and restaurant recommendations for major cities you might want to visit after Paris:

### Proposed Itinerary (After Paris)

---

### Day 4-5: Amsterdam, Netherlands
- **Transportation**: Train from Paris to Amsterdam (Approx. 3.5 hours)

#### **Flights from Paris to Amsterdam**
- **Airline**: Thalys train
- **Cost**: Approx. €50-€120

#### **Accommodations**
1. **[Stayokay Amsterdam Vondelpark](https://www.stayokay.com/en/hostels/amsterdam-vondelpark)**
   - Price: ~$40/night (hostel)
   - Rating: 4.3/5

2. **[NH Collection Amsterdam Flower Market](https://www.nh-hotels.com/hotel/nh-collection-amsterdam-flower-market)**
   - Price: ~$200/night
   - Rating: 4.5/5

#### **Restaurants**
- **[De Silveren Spiegel](https://www.yelp.com/biz/de-silveren-spiegel-amsterdam?osq=restaurants)**: 
  - Cuisine: Dutch
  - Rating: 4.5/5

---

### Day 6-7: Berlin, Germany
- **Transportation**: Flight or train from Am

In [34]:
new_prompt = """will not you arrange flight tickets, hotels and restaurants for all your recommendations?"""
new_input = assemble_conversation(result, new_prompt)
result = await Runner.run(travel_agent, new_input)
print(result.final_output)

Error getting response: Error code: 400 - {'error': {'message': 'Your input exceeds the context window of this model. Please adjust your input and try again.', 'type': 'invalid_request_error', 'param': 'input', 'code': 'context_length_exceeded'}}. (request_id: req_70e030ab09a6555438c4e86a4ae363f4)


BadRequestError: Error code: 400 - {'error': {'message': 'Your input exceeds the context window of this model. Please adjust your input and try again.', 'type': 'invalid_request_error', 'param': 'input', 'code': 'context_length_exceeded'}}

In [ ]:
for item in orchestrator_result.new_items:
            if isinstance(item, MessageOutputItem):
                text = ItemHelpers.text_message_output(item)
                if text:
                    print(f"  - Translation step: {text}")

In [84]:
def yelp_search(search_term:str, location:str ):
    params = {
    "engine": "yelp",
    "find_desc": search_term,
    "find_loc": location,
    "api_key": api_key
    }

    search = GoogleSearch(params)
    results = search.get_dict()
    organic_results = results["organic_results"]
    return organic_results

In [86]:
yelp_res = yelp_search(search_term='breakfast', location='London')

In [90]:
from typing import List, Optional
from pydantic import BaseModel, HttpUrl, Field

class Category(BaseModel):
    """Model for business categories"""
    title: str
    link: HttpUrl

class Business(BaseModel):
    """Main model for Yelp business listings with all optional fields accounted for"""
    position: int
    place_ids: List[str]
    title: str
    link: HttpUrl
    reviews_link: HttpUrl
    categories: List[Category]
    
    # Made optional based on error
    price: Optional[str] = Field(None, description="Price indicator like '££'")  
    rating: Optional[float] = None
    reviews: Optional[int] = None
    neighborhoods: Optional[str] = None
    phone: Optional[str] = None
    snippet: Optional[str] = None
    thumbnail: Optional[HttpUrl] = None
    
    # Already optional
    highlights: Optional[List[str]] = None

class YelpSearchResults(BaseModel):
    """Container for the top-level list"""
    businesses: List[Business]

In [91]:
results = YelpSearchResults(businesses=yelp_res)

In [95]:
# from yelp_models import YelpSearchResults  # Assuming you have this model defined

# Assuming 'data' is your YelpSearchResults instance
for business in results.businesses:
    categories = ', '.join([category.title for category in business.categories])
    highlights = ', '.join(business.highlights) if business.highlights else 'None'
    price = business.price if business.price else 'N/A'
    
    print(f"""
{business.position}. {business.title} ({business.neighborhoods})
   📍 Categories: {categories}
   💷 Price: {price}
   ⭐ Rating: {business.rating} ({business.reviews} reviews)
   📞 Phone: {business.phone}
   🔗 Link: {business.link}
   🖼️ Thumbnail: {business.thumbnail}
   ✨ Highlights: {highlights}
   📝 Snippet: {business.snippet}
""")



1. Drury 188-189 (Covent Garden)
   📍 Categories: Breakfast & Brunch
   💷 Price: ££
   ⭐ Rating: 4.8 (61 reviews)
   📞 Phone: 020 7404 7664
   🔗 Link: https://www.yelp.com/biz/drury-188-189-london-3?osq=breakfast
   🖼️ Thumbnail: https://s3-media0.fl.yelpcdn.com/bphoto/fc5JU7mwe6gfdGHMKqJLyg/348s.jpg
   ✨ Highlights: None
   📝 Snippet: Delicious breakfast with fast and efficient service. Really love the toast options with adding your...


2. The Breakfast Club (Soho)
   📍 Categories: Coffee & Tea, Breakfast & Brunch, American
   💷 Price: ££
   ⭐ Rating: 3.9 (605 reviews)
   📞 Phone: 020 7434 2571
   🔗 Link: https://www.yelp.com/biz/the-breakfast-club-london-2?osq=breakfast
   🖼️ Thumbnail: https://s3-media0.fl.yelpcdn.com/bphoto/HTreqr7I1eGQTARGzuC1iQ/348s.jpg
   ✨ Highlights: Good for Breakfast
   📝 Snippet: Great breakfast spot! Delicious eggs!! We enjoyed our breakfast and it's got a cute vibe!


3. Sandwich Street Kitchen (Bloomsbury)
   📍 Categories: Cafes
   💷 Price: N/A
   ⭐ Ra

In [5]:
def hotels_search(q:str, check_in_date:str, check_out_date:str, gl:str):
    """
    Google hotels information from SERPAPI. Returns json structure
    q : Search Query
    gl : Country
    """
    params = {
            "engine": "google_hotels",
            "q": q,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "2",
            "currency": "USD",
            "gl": gl,
            "hl": "en",
            "api_key": "06308305345bc9d73b8207f48b29d3f871e7a6c28e886739d62591f264da5366"
            }

    search = GoogleSearch(params)
    results = search.get_dict()

    return results

In [6]:
hotel_res = hotels_search(q="london", check_in_date= "2025-05-14", check_out_date="2025-05-23", gl='us')

In [10]:
from typing import List, Optional, Dict, Union
from pydantic import BaseModel, HttpUrl


class Transportation(BaseModel):
    type: str
    duration: str


class NearbyPlace(BaseModel):
    name: str
    transportations: List[Transportation]


class Rate(BaseModel):
    lowest: str
    extracted_lowest: int
    before_taxes_fees: Optional[str] = None
    extracted_before_taxes_fees: Optional[int] = None


class Price(BaseModel):
    source: str
    logo: HttpUrl
    num_guests: int
    rate_per_night: Rate
    free_cancellation: Optional[bool] = None  # Made optional
    free_cancellation_until_date: Optional[str] = None
    free_cancellation_until_time: Optional[str] = None


class Image(BaseModel):
    thumbnail: HttpUrl
    original_image: HttpUrl


class Rating(BaseModel):
    stars: int
    count: int


class ReviewsBreakdown(BaseModel):
    name: str
    description: str
    total_mentioned: int
    positive: int
    negative: int
    neutral: int


class GPSCoordinates(BaseModel):
    latitude: float
    longitude: float


class Property(BaseModel):
    type: str
    name: str
    description: Optional[str] = None
    link: Optional[HttpUrl] = None  # Made optional
    property_token: str
    serpapi_property_details_link: HttpUrl
    gps_coordinates: GPSCoordinates
    check_in_time: Optional[str] = None
    check_out_time: str
    rate_per_night: Rate
    total_rate: Rate
    deal: Optional[str] = None
    deal_description: Optional[str] = None
    nearby_places: List[NearbyPlace]
    hotel_class: Optional[str] = None
    extracted_hotel_class: Optional[int] = None
    images: List[Image]
    overall_rating: Optional[Union[float, int]] = None  # Made optional
    reviews: Optional[int] = None  # Made optional
    ratings: Optional[List[Rating]] = None  # Made optional
    location_rating: Union[float, int]
    reviews_breakdown: Optional[List[ReviewsBreakdown]] = None
    amenities: List[str]
    eco_certified: Optional[bool] = False
    excluded_amenities: Optional[List[str]] = None
    essential_info: Optional[List[str]] = None
    prices: Optional[List[Price]] = None


class BrandChild(BaseModel):
    id: int
    name: str


class Brand(BaseModel):
    id: int
    name: str
    children: List[BrandChild] = []


class SearchMetadata(BaseModel):
    id: str
    status: str
    json_endpoint: HttpUrl
    created_at: str
    processed_at: str
    google_hotels_url: HttpUrl
    raw_html_file: HttpUrl
    prettify_html_file: HttpUrl
    total_time_taken: float


class SearchParameters(BaseModel):
    engine: str
    q: str
    gl: str
    hl: str
    currency: str
    check_in_date: str
    check_out_date: str
    adults: int
    children: int


class SearchInformation(BaseModel):
    total_results: int


class SerpapiPagination(BaseModel):
    current_from: int
    current_to: int
    next_page_token: str
    next: HttpUrl


class HotelSearchResponse(BaseModel):
    search_metadata: SearchMetadata
    search_parameters: SearchParameters
    search_information: SearchInformation
    brands: List[Brand]
    properties: List[Property]
    serpapi_pagination: SerpapiPagination


# Example usage (assuming 'data' is your dictionary)
# response = HotelSearchResponse(**data)
# print(response.search_metadata.id)
# print(response.properties[0].name)

In [11]:
response = HotelSearchResponse(**hotel_res)

type='hotel' name='Hyde Park Green Hotel' description=None link=HttpUrl('https://www.hydeparkgreen.london/') property_token='ChkIx4zF2dLiyNNuGg0vZy8xMXFiOGZkcDkzEAE' serpapi_property_details_link=HttpUrl('https://serpapi.com/search.json?adults=2&check_in_date=2025-05-14&check_out_date=2025-05-23&children=0&currency=USD&engine=google_hotels&gl=us&hl=en&property_token=ChkIx4zF2dLiyNNuGg0vZy8xMXFiOGZkcDkzEAE&q=london') gps_coordinates=GPSCoordinates(latitude=51.512638599999995, longitude=-0.1868081) check_in_time=None check_out_time='11:00\u202fAM' rate_per_night=Rate(lowest='$186', extracted_lowest=186, before_taxes_fees='$152', extracted_before_taxes_fees=152) total_rate=Rate(lowest='$1,672', extracted_lowest=1672, before_taxes_fees='$1,369', extracted_before_taxes_fees=1369) deal='25% less than usual' deal_description='Deal' nearby_places=[NearbyPlace(name='Bayswater', transportations=[Transportation(type='Walking', duration='2 min')]), NearbyPlace(name='Heathrow Airport', transportati

In [36]:
for i in response.properties:
    print(f"name:{i.name}, vacation type :{i.type}, price_per_night: {i.rate_per_night.lowest}")

name:Hyde Park Green Hotel, vacation type :hotel, price_per_night: $186
name:The Rose & Crown, London Bridge (PUBLOVE), vacation type :hotel, price_per_night: $90
name:Big Home in London, Forest Hill, vacation type :vacation rental, price_per_night: $91
name:Thistle London Hyde Park Kensington Gardens, vacation type :hotel, price_per_night: $181
name:Lovely apartment-12 minutes tube from Central London,sleeps 6,parking on street, vacation type :vacation rental, price_per_night: $146
name:Luxurious 4-Bedrooms Apartment in Central London, vacation type :vacation rental, price_per_night: $104
name:Wilde Aparthotels, London, Covent Garden, vacation type :hotel, price_per_night: $290
name:London Backpackers Hostel, vacation type :hotel, price_per_night: $29
name:Hyatt Regency London Albert Embankment, vacation type :hotel, price_per_night: $292
name:Lord Kensington Hotel, vacation type :hotel, price_per_night: $123
name:Radisson Blu Hotel, London South Kensington, vacation type :hotel, pric